In [1]:
import pickle
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torchvision
import matplotlib.pyplot as plt
import sinabs
import sinabs.activation
import sinabs.layers as sl
from sinabs.from_torch import from_model
import os
import shutil
from sklearn.model_selection import KFold
print(torch.cuda.is_available())
torch.cuda.get_device_name(0)

True


'NVIDIA GeForce RTX 2080 Ti'

Data

In [2]:
class MyData(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.data_path = os.listdir(self.root_dir)
    
    def __getitem__(self, idx):
        data_name = self.data_path[idx]
        data_item_path = os.path.join(self.root_dir, data_name)
        with open(data_item_path, 'rb') as f:
            data = np.load(f)

        data = torch.from_numpy(data).float()
        data = torch.transpose(data, 0, 1)

        label = torch.tensor(eval(data_name[-5]), dtype=torch.long)
        
        return data, label

    def __len__(self):
        return len(self.data_path)

root_dir_1 = 'F:\Files\PhD\Braille\Data/braille-27letters-sphere/effect-depth/train'
root_dir_2 = 'F:\Files\PhD\Braille\Data/braille-27letters-sphere/effect-depth/test'

train_data = MyData(root_dir_1)
test_data  = MyData(root_dir_2)

In [3]:
train_data[0][0].shape

torch.Size([720, 100])

Training

In [ ]:
lr = 1e-4

device = 'cuda:0'
k_folds = 10
kfold = KFold(n_splits=k_folds, shuffle=False)
batch_size = 81
loss_fn = nn.CrossEntropyLoss()

fold_train_acc = []
fold_val_acc = []

for fold, (train_ids, val_ids) in enumerate(kfold.split(train_data)):
    # 模型初始化
    linear_model = nn.Sequential(
        nn.Linear(100, 400),
        nn.ReLU(),
        nn.Linear(400, 27)
    )
    linear_model = from_model(linear_model, batch_size=81, input_shape=(1, 700, 100), 
                              add_spiking_output=True).to(device)
    optimizer = torch.optim.Adam(linear_model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    # 分割数据集
    train_sub = Subset(train_data, train_ids)
    val_sub = Subset(train_data, val_ids)
    # 创建数据加载器
    train_loader = DataLoader(train_sub, batch_size, shuffle=True)
    val_loader  = DataLoader(val_sub, batch_size)

    # 训练模型
    linear_model.train()
    epochs = 40
    for e in range(epochs):
        running_loss = 0.
        acc = 0
        scheduler.step()
        for i, (input, target) in enumerate(train_loader):
            optimizer.zero_grad()
            linear_model.reset_states()

            input = input.to(device)
            target = target.to(device)
            output = linear_model(input)
            sum_output = output.sum(1)
            loss = loss_fn(sum_output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss

            for j in range(batch_size):
                if sum_output[j].argmax() == target[j]:
                    acc += 1

        print("epoch: %d, accuracy: %.2f%%, running_loss: %.2f, current_lr: %.6f" 
              % (e, acc/len(train_sub)*100, running_loss, scheduler.get_last_lr()[0]) )
    fold_train_acc.append(np.around(acc/len(train_sub)*100, 2))

    # 验证模型
    acc_num = 0
    for i, (data, target) in enumerate(val_loader):
        with torch.no_grad():
            linear_model.reset_states()
            data = data.to(device)
            target = target.to(device)
            output = linear_model(data)
            sum_output = output.sum(1)

        for j in range(batch_size):
            if sum_output[j].argmax() == target[j]:
                acc_num += 1
    print("accuracy on validation set: %.2f%%" % (acc_num/len(val_sub)*100))
    fold_val_acc.append(np.around(acc_num/len(val_sub)*100, 2))

In [ ]:
print(fold_train_acc)
print(fold_val_acc)

sum = 0
cnt = 0
for i in fold_train_acc:
    sum += i
    cnt += 1
print(sum/cnt)

sum = 0
cnt = 0
for i in fold_val_acc:
    sum += i
    cnt += 1
print(sum/cnt)

In [6]:
model_path = './models/stepA.pth'
torch.save(linear_model, model_path)

Testing

In [ ]:
model = torch.load(model_path)
test_loader  = DataLoader(test_data, batch_size)

acc_num = 0
for i, (data, target) in enumerate(test_loader):
    with torch.no_grad():
        model.reset_states()
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        sum_output = output.sum(1)

    for j in range(batch_size):
        if sum_output[j].argmax() == target[j]:
            acc_num += 1
print("accuracy on testing set: %.2f%%" % (acc_num/len(test_data)*100))